In [7]:
import os
import random
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

In [ ]:
random.seed =40
np.random.seed(40)
tf.random.set_seed(40)


In [2]:
# Define paths
labels_path = "D:\\dataset for lab programs\\labels_traffic-data"
images_path = "D:\\dataset for lab programs\\traffic data"

In [3]:
# Load data
image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg') or f.endswith('.png')]
label_files = [f for f in os.listdir(labels_path) if f.endswith('.txt')]

In [4]:
print(f"Number of images: {len(image_files)}")

Number of images: 9


In [6]:
# Preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def preprocess_data(image_files, label_files):
    X = []
    y = []

    # Define ImageDataGenerator for data augmentation
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest'
    )
    for image_file, label_file in zip(image_files, label_files):
        # Resize images to a standardized input size
        image = cv2.imread(os.path.join(images_path, image_file))
        image = cv2.resize(image, (416, 416))
        X.append(image)

        # Convert annotations to YOLO format
        with open(os.path.join(labels_path, label_file), 'r') as f:
            lines = f.readlines()
            for line in lines:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())
                x_center /= image.shape[1]
                y_center /= image.shape[0]
                width /= image.shape[1]
                height /= image.shape[0]
                y.append([class_id, x_center, y_center, width, height])

    # Split the dataset into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    return X_train, X_val, X_test, y_train, y_val, y_test

In [8]:
def yolo_model(input_shape,num_classes):
    input_layer = Input(shape=input_shape)

    # Convolutional layers
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    # Fully connected layers
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    output_layer = Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    return model

In [ ]:
# Training
def train_model(X_train, y_train, X_val, y_val):
    model = yolo_model(num_classes=1)  # Assuming a single class for vehicles
    model.compile(optimizer=Adam(lr=0.001), loss='mse', metrics=['accuracy'])

    checkpoint = ModelCheckpoint('yolo_model.h5', monitor='val_loss', verbose=1, save_best_only=True)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

    model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=50,
              batch_size=32,
              callbacks=[checkpoint, early_stopping])

    return model